In [1]:
import pyodbc 
import tkinter as tk
from tkinter import *
from tkinter import ttk, filedialog
import ttkbootstrap as ttkbs
from ttkbootstrap.constants import *
import cv2
import numpy as np
from PIL import Image, ImageTk

imagem_global = None
imagem_sobel = None  # Variável global para armazenar a imagem com filtro Sobel

def abrir_imagem():
    global imagem_global, imagem_sobel  # Declara as variáveis globais
    caminho_imagem = filedialog.askopenfilename(title="Escolha uma imagem", 
                                                filetypes=[("Arquivos de imagem", "*.jpg;*.png;*.jpeg")])
    if caminho_imagem:
        img = Image.open(caminho_imagem)
        img.thumbnail((400, 400))
        img_tk = ImageTk.PhotoImage(img)
        lbl_imagem.config(image=img_tk)
        lbl_imagem.image = img_tk  # Necessário para o tkinter manter a referência
        imagem_global = np.array(img)  # Converte a imagem PIL para NumPy

        # Aplica o filtro Sobel imediatamente após carregar a imagem
        aplicar_filtro('xy')

        # Mostra os botões após carregar a imagem
        btn_frame.pack(pady=10)

def aplicar_filtro(filtro):
    global imagem_global, imagem_sobel
    if imagem_global is not None:
        imagem_gray = cv2.cvtColor(imagem_global, cv2.COLOR_RGB2GRAY)
        if filtro == 'x':
            sobel = cv2.Sobel(imagem_gray, cv2.CV_64F, 1, 0, ksize=3)
        elif filtro == 'y':
            sobel = cv2.Sobel(imagem_gray, cv2.CV_64F, 0, 1, ksize=3)
        elif filtro == 'xy':  # Aplica ambos os filtros e combina
            sobel_x = cv2.Sobel(imagem_gray, cv2.CV_64F, 1, 0, ksize=3)
            sobel_y = cv2.Sobel(imagem_gray, cv2.CV_64F, 0, 1, ksize=3)
            sobel = np.sqrt(sobel_x**2 + sobel_y**2)

        sobel_normalizado = np.uint8(255 * (sobel - np.min(sobel)) / (np.max(sobel) - np.min(sobel)))
        imagem_sobel = Image.fromarray(sobel_normalizado)
        imagem_sobel.thumbnail((400, 400))
        img_sobel_tk = ImageTk.PhotoImage(imagem_sobel)
        
        lbl_imagem_sobel.config(image=img_sobel_tk)
        lbl_imagem_sobel.image = img_sobel_tk

def remover_filtro():
    global imagem_global
    if imagem_global is not None:
        img_tk = ImageTk.PhotoImage(Image.fromarray(imagem_global))
        lbl_imagem_sobel.config(image=img_tk)
        lbl_imagem_sobel.image = img_tk  # Atualiza a imagem

def aplicar_filtro_xy():
    aplicar_filtro('xy')  # Reaplica o filtro Sobel XY

# Cria a janela principal
janela = ttkbs.Window(themename="darkly")
janela.title("Tela de Seleção de Imagem")
janela.configure(bg="#333")

# Define o tamanho da janela e a centraliza
janela.geometry("900x500")
largura_tela = janela.winfo_screenwidth()
altura_tela = janela.winfo_screenheight()
pos_x = (largura_tela // 2) - (900 // 2)
pos_y = (altura_tela // 2) - (500 // 2)
janela.geometry(f"900x500+{pos_x}+{pos_y}")

menu_principal = Menu(janela)
janela.config(menu=menu_principal)

menu_opcoes = Menu(menu_principal, tearoff=0, bg="#1f4b99", fg="white", activebackground="#4a6fa5", activeforeground="white")
menu_principal.add_cascade(label="Menu", menu=menu_opcoes)
menu_opcoes.add_command(label="Add imagem", command=abrir_imagem)
menu_opcoes.add_command(label="Sair", command=janela.destroy)

# Layout principal
frame = tk.Frame(janela, bg="#333")
frame.pack(pady=20)

lbl_imagem = tk.Label(frame, bg="#333")
lbl_imagem.grid(row=0, column=0, padx=(20, 10))  # Imagem original

lbl_imagem_sobel = tk.Label(frame, bg="#333")
lbl_imagem_sobel.grid(row=0, column=1, padx=(10, 20))  # Imagem com filtro

# Botões para aplicar filtros e remover filtro (inicialmente ocultos)
btn_frame = tk.Frame(janela, bg="#333")

btn_sobel_x = tk.Button(btn_frame, text="Filtro Sobel em X", command=lambda: aplicar_filtro('x'), bg="#1f4b99", fg="white")
btn_sobel_x.pack(side=tk.LEFT, padx=5)

btn_sobel_y = tk.Button(btn_frame, text="Filtro Sobel em Y", command=lambda: aplicar_filtro('y'), bg="#1f4b99", fg="white")
btn_sobel_y.pack(side=tk.LEFT, padx=5)

btn_sobel_xy = tk.Button(btn_frame, text="Filtro Sobel XY", command=aplicar_filtro_xy, bg="#1f4b99", fg="white")
btn_sobel_xy.pack(side=tk.LEFT, padx=5)

btn_remover_filtro = tk.Button(btn_frame, text="Remover Filtro", command=remover_filtro, bg="#1f4b99", fg="white")
btn_remover_filtro.pack(side=tk.LEFT, padx=5)

# Inicia o loop da interface gráfica
janela.mainloop()
